# 第10章
---
# 时间序列

不管在哪个领域中，时间序列（time series）数据都是一种重要的结构化数据形式。在多个时间点观察或测量到的任何事物都可以形成一段时间序列。时间序列有固定频率的，也可以是不定期的。数据的意义取决于具体的应用场景，主要有以下几种：

- 时间戳（timestamp），特定的时刻
- 固定时期（period），如2007年1月或2010年全年
- 时间间隔（interval），由起始和结束时间戳表示。时期（period）可以被看做间隔（interval）的特例
- 实验或过程时间，每个时间点都是相对于特定起始时间的一个度量。例如，从放入烤箱时起，每秒钟饼干的直径。

本章主要讲解前三种时间序列，timestamp、period、interval。许多技术都可以用于处理实验型时间序列，其索引可能是一个整数或浮点数（表示从实验开始算起已经过去的时间）。最简单也最常见的时间序列都是用时间戳进行索引的。

pandas提供的时间序列处理工具和数据算法：轻松的切片/切块、聚合、对定期/不定期的时间序列进行重采样等。对金融和经济数据尤为游泳，当然也可以分析任何带时间序列的数据集，如服务器日志数据。

## 日期和时间数据类型及工具

Python标准库包含用于日期（date）和时间（time）数据的数据类型，且有日历方面的功能。主要用到**datetime、time、以及calendar模块**。datetime.datetime是用的最多的数据类型：

In [1]:
from datetime import datetime

now = datetime.now()
now

datetime.datetime(2016, 7, 11, 16, 55, 43, 875000)

In [2]:
now.year, now.month, now.day

(2016, 7, 11)

datetime以毫秒形式存储日期和时间。datetime.timedelta表示两个datetime对象之间的时间差：

In [3]:
delta = datetime(2011, 1, 7) - datetime(2008, 6, 24, 8, 15)
delta

datetime.timedelta(926, 56700)

In [4]:
delta.days

926

In [5]:
delta.seconds

56700

可以给datetime对象加上（或减去）一个或多个timedelta，这样会产生一个新对象：

In [6]:
from datetime import timedelta

start = datetime(2016, 1, 7)
start + timedelta(10)

datetime.datetime(2016, 1, 17, 0, 0)

In [7]:
start - 2 * timedelta(11, 11)

datetime.datetime(2015, 12, 15, 23, 59, 38)

Python datetime的数据类型

类型 | 说明
- | -
date | 以公历形式存储日历日期（年、月、日）
time | 将时间存储为时、分、秒、毫秒
datetime | 存储日期和时间
timedelta | 表示两个datetime值之间的差（日、秒、毫秒）

### 字符串和datetime的相互转换
利用str或strftime方法（传入一个格式化字符串），datetime对象和pandas的Timestamp对象可以被格式化为字符串：

In [8]:
stamp = datetime(2011, 1, 3)

str(stamp)

'2011-01-03 00:00:00'

In [9]:
stamp.strftime('%Y-%m-%d')

'2011-01-03'

In [10]:
value = '2016-07-05'

datetime.strptime(value, '%Y-%m-%d')

datetime.datetime(2016, 7, 5, 0, 0)

In [11]:
datestrs = ['7/11/2016', '7/15/2016']
[datetime.strptime(x, '%m/%d/%Y') for x in datestrs]

[datetime.datetime(2016, 7, 11, 0, 0), datetime.datetime(2016, 7, 15, 0, 0)]

datetime.strptime是通过已知格式进行日期解析的最佳方式。但是每次都要编写格式定义是很麻烦的事情，尤其一些常见的格式。这样可以用deteutil这个三方包中的parser.parse方法：

In [13]:
from dateutil.parser import parse

parse('2011/07/19')

datetime.datetime(2011, 7, 19, 0, 0)

In [14]:
# dateutil可以寄解析几乎所有人能够理解的日期形式：（中文好像不行）
parse('Jan 31, 1997 10:22 PM')

datetime.datetime(1997, 1, 31, 22, 22)

In [16]:
# 国际通用格式中，经常有日在前的，传入dayfirst=True即可：
parse('6/12/2016', dayfirst=True)

datetime.datetime(2016, 12, 6, 0, 0)

pandas通常是用于处理成组日期的，不管是DataFrame的轴索引还是列。to_datetime的方法可以解析多种不同的日期表示形式。对标准日期如ISO8601的解析非常快。

In [17]:
datestrs

['7/11/2016', '7/15/2016']

In [19]:
import pandas as pd
pd.to_datetime(datestrs)

DatetimeIndex(['2016-07-11', '2016-07-15'], dtype='datetime64[ns]', freq=None)

In [20]:
# 处理缺失值（None、空字符串等）:
idx = pd.to_datetime(datestrs + [None])
idx

DatetimeIndex(['2016-07-11', '2016-07-15', 'NaT'], dtype='datetime64[ns]', freq=None)

In [23]:
idx[2] # not a time pandas中时间戳数据的NA值

NaT

In [22]:
pd.isnull(idx)

array([False, False,  True], dtype=bool)

**警告：**dateutil.parser会把一些不是日期的字符串认作是日期（比如“42”会被解析为2042年的今天）。

datetime格式定义（兼容ISO C89）

代码 | 说明
- | -
%Y | 4位数的年
%y | 2位数的年
%m | 2位数的月[01,12]
%d | 2位数的日[01,31]
%H | 时（24小时制）[00,23]
%I | 时（12小时制）[01,12]
%M | 2位数的分[00,59]
%S | 秒[00,61]（秒60和秒61用于闰秒）
%w | 用整数表示的星期几[0(星期天)，6]
%U | 每年的第几周[00,53]。星期天被任务是每周的第一天，每年第一个星期天之前的那几天被任务是“第0周”
%W | 每年的第几周[00,53]。星期一被任务是每周的第一天，每年第一个星期一之前的那几天被任务是“第0周”
%z | 以+HHMM或-HHMM表示的UTC时区偏移量，如果时间对象为naive，则返回空字符串
%F | %Y-%m-%d简写形式，例如2016-7-11
%D | %m/%d/%y简写形式，例如11/07/12